In [1]:
def fibonacci(n, cache=None):
    if cache is None: 
        cache = {}
    
    if n in cache:
        return cache[n]
    if n <= 1:
        return 1
    cache[n] = fibonacci(n - 1, cache) + fibonacci(n - 2, cache)
    return cache[n]


In [6]:
timeit fibonacci(50) #极大提高运行效率

10000 loops, best of 3: 28.5 µs per loop


In [20]:
# 使用装饰器简化，没必要手动为每个函数添加代码
def memo(func):
    print('__in_decorator__')
    cache = {} #闭包
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

def fibonacci(n):
    if n <= 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [21]:
fibonacci = memo(fibonacci)
print(fibonacci(50))

__in_decorator__
20365011074


In [22]:
#使用@语法糖
@memo
def fibonacci(n):
    if n <= 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)
print(fibonacci(50))

__in_decorator__
20365011074


In [23]:
@memo
def climb(n, steps):
    count = 0
    if n==0:
        count = 1
    elif n > 0:
        for step in steps:
            count += climb(n - step, steps)
    return count

__in_decorator__


In [24]:
climb(10, (1,2,3))

274